In [1]:
import logging
import os
import random
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.CepsMy1Experiment import CepsMy1Experiment
from src.experiment.PcitExperiment import PcitExperiment, _Clustering
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.experiment.CepsExperiment import CepsExperiment
pd.options.display.max_columns = 999

In [18]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

In [3]:
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance
t_c = 300
K = 2
n = 2
t_ini = train_instances.size * 10 * 10
t_i = 10
max_iter = 1

experiment = CepsMy1Experiment(
    t_c=t_c,
    t_ini=t_ini,
    t_i=t_i,
    K=K,
    n=n,
    max_iter=max_iter,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-12-14 23:10:57] INFO      [CEPS_MY_1] Start!
[2024-12-14 23:10:58] INFO      Initialization
[2024-12-14 23:10:58] INFO      Solver 1/2
[2024-12-14 23:10:58] INFO      Random config 1/10
[2024-12-14 23:10:58] DEBUG     =================================  Portfolio  ==================================
[2024-12-14 23:10:58] DEBUG     2220588739511820280 --> ASCENT_CANDIDATES=52;BACKBONE_TRIALS=1;BACKTRACKING=YES;CANDIDATE_SET_TYPE=ALPHA;EXTRA_CANDIDATES=7;EXTRA_CANDIDATE_SET_TYPE=QUADRANT;GAIN23=NO;GAIN_CRITERION=YES;INITIAL_STEP_SIZE=1;INITIAL_TOUR_ALGORITHM=QUICK-BORUVKA;INITIAL_TOUR_FRACTION=0.5820197920751;KICKS=3;KICK_TYPE=0;MAX_BREADTH=954863081;MAX_CANDIDATES=10;MOVE_TYPE=3;PATCHING_A=3;PATCHING_C=1;POPULATION_SIZE=8;RESTRICTED_SEARCH=YES;SUBGRADIENT=YES;SUBSEQUENT_MOVE_TYPE=6;SUBSEQUENT_PATCHING=NO
[2024-12-14 23:10:58] DEBUG     ================================================================================
[2024-12-14 23:10:58] DEBUG     executor start
[2024-12-14 23:11:06]

In [ ]:
# t_c = 60
# K = 2
# n = 2
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = GlobalExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 50
# K = 2
# n = 2
# max_iter = 2
# solver_class = TSP_LKH_Solver
# instance_class = TSP_Instance

# experiment = PcitExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 50
# K = 2
# n = 2
# max_iter = 2
# solver_class = TSP_LKH_Solver
# instance_class = TSP_Instance

# experiment = PcitExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance
# t_c = 200
# K = 2
# n = 2
# t_ini = train_instances.size * 10 * 2
# t_i = 10
# max_iter = 2

# experiment = CepsExperiment(
#     t_c=t_c,
#     t_ini=t_ini,
#     t_i=t_i,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance
# t_c = 200
# K = 2
# n = 2
# t_ini = train_instances.size * 10 * 2
# t_i = 10
# max_iter = 1

# experiment = CepsMy1Experiment(
#     t_c=t_c,
#     t_ini=t_ini,
#     t_i=t_i,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [22]:
# remaining_time = np.ones(shape=(K, )) * np.inf
# best_portfolio.evaluate(test_instances, remaining_time, "test")

In [42]:
from src.database import db_connect
conn = db_connect()

In [2]:
import sqlite3
# conn = sqlite3.connect("_archive/7/CEPS1.db")
conn = sqlite3.connect("database/2024_12_15_17_10_07.db")

In [3]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,INITIAL_TOUR_ALGORITHM,INITIAL_TOUR_FRACTION,KICKS,KICK_TYPE,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,2152573642869061282,41,0,YES,ALPHA,3,QUADRANT,YES,YES,1,BORUVKA,0.636567,0,0,1715235503,4,2,3,5,18,YES,YES,5,NO
1,686356820175302440,53,0,YES,QUADRANT,2,QUADRANT,YES,YES,5,WALK,0.369952,5,5,2143548490,6,4,0,1,27,NO,NO,4,NO
2,508943627851605191,41,0,NO,DELAUNAY,2,QUADRANT,NO,YES,1,BORUVKA,0.621888,4,0,1323058798,5,4,2,1,65,YES,YES,3,NO
3,1191837438205487429,58,0,NO,DELAUNAY,8,QUADRANT,NO,YES,3,GREEDY,0.185182,0,0,82192709,7,2,4,5,81,YES,NO,3,NO


In [4]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp
1,data/TSP/CEPS_benchmark/grid/04.tsp
2,data/TSP/CEPS_benchmark/grid/33.tsp
3,data/TSP/CEPS_benchmark/rotation/45.tsp
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp
5,data/TSP/CEPS_benchmark/cluster/15.tsp
6,data/TSP/CEPS_benchmark/implosion/16.tsp
7,data/TSP/CEPS_benchmark/uniform_portgen/39.tsp
8,data/TSP/CEPS_benchmark/expansion/09.tsp
9,data/TSP/CEPS_benchmark/expansion/03.tsp


In [5]:
df = pd.read_sql_query("SELECT * FROM results", conn)
df

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,2152573642869061282,1.56,1.56,test1,2024-12-15 16:10:10
1,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,686356820175302440,7.99,7.99,test1,2024-12-15 16:10:17
2,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,508943627851605191,100.00,10.00,test1,2024-12-15 16:10:19
3,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,1191837438205487429,1.20,1.20,test1,2024-12-15 16:10:19
4,data/TSP/CEPS_benchmark/grid/04.tsp,2152573642869061282,1.17,1.17,test1,2024-12-15 16:10:19
...,...,...,...,...,...,...
115,data/TSP/CEPS_benchmark/cluster_netgen/46.tsp,1191837438205487429,0.57,0.57,test1,2024-12-15 16:12:34
116,data/TSP/CEPS_benchmark/explosion/33.tsp,2152573642869061282,0.83,0.83,test1,2024-12-15 16:12:34
117,data/TSP/CEPS_benchmark/explosion/33.tsp,686356820175302440,100.00,10.00,test1,2024-12-15 16:12:37
118,data/TSP/CEPS_benchmark/explosion/33.tsp,508943627851605191,4.80,4.80,test1,2024-12-15 16:12:37


In [6]:
df["comment"].value_counts()

test1    120
Name: comment, dtype: int64

In [7]:
_ = df.loc[df["comment"].str.contains("test")].pivot_table(index=["instance_id", "solver_id"], columns="comment", values="time").sort_values(["instance_id", "solver_id"])
_["range"] = _.max(axis=1) - _.min(axis=1)
_.sort_values(by="range")

comment                                                              test1  \
instance_id                                     solver_id                    
data/TSP/CEPS_benchmark/cluster/15.tsp          1191837438205487429  10.00   
data/TSP/CEPS_benchmark/linearprojection/05.tsp 686356820175302440   10.00   
                                                508943627851605191    2.65   
                                                2152573642869061282   5.79   
                                                1191837438205487429   2.84   
...                                                                    ...   
data/TSP/CEPS_benchmark/expansion/04.tsp        2152573642869061282   0.87   
                                                1191837438205487429  10.00   
data/TSP/CEPS_benchmark/expansion/03.tsp        686356820175302440   10.00   
data/TSP/CEPS_benchmark/uniform_portgen/39.tsp  508943627851605191   10.00   
                                                686356820175302440   10.00   

comment                                                              range  
instance_id                                     solver_id                   
data/TSP/CEPS_benchmark/cluster/15.tsp          1191837438205487429    0.0  
data/TSP/CEPS_benchmark/linearprojection/05.tsp 686356820175302440     0.0  
                                                508943627851605191     0.0  
                                                2152573642869061282    0.0  
                                                1191837438205487429    0.0  
...                                                                    ...  
data/TSP/CEPS_benchmark/expansion/04.tsp        2152573642869061282    0.0  
                                                1191837438205487429    0.0  
data/TSP/CEPS_benchmark/expansion/03.tsp        686356820175302440     0.0  
data/TSP/CEPS_benchmark/uniform_portgen/39.tsp  508943627851605191     0.0  
                                                686356820175302440     0.0  

[120 rows x 2 columns]

In [ ]:
_ = df.loc[df["comment"].str.startswith("test")].pivot_table(index="instance_id", columns="solver_id", values="cost", aggfunc=lambda x: sorted(x)[1])
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

In [ ]:
_["min"].describe()

In [ ]:
(_["min"] > 90).value_counts()

In [ ]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

In [ ]:
_.sort_values(by="min")

In [ ]:
# config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
# config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
# solver = TSP_LKH_Solver(config=config)

# instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

# solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index